In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import text
import time
from io import StringIO
from google.cloud import storage
import datetime
from google.cloud import bigquery
import pytz
from pandas_gbq import to_gbq

In [2]:
#setup big query 
client = bigquery.Client()

In [3]:
#function to load dataframe to bigquery
def load_dataframe_to_bigquery(df, dataset_id, table_id, project_id, if_exists='replace'):
    """
    Load a DataFrame to a Google BigQuery table and return a status message.

    Parameters:
    df (DataFrame): The pandas DataFrame to load.
    dataset_id (str): The dataset ID in BigQuery.
    table_id (str): The table ID within the dataset.
    project_id (str): Your Google Cloud project ID.
    if_exists (str): Action to take if the table already exists. Options are 'fail', 'replace', 'append'.

    Returns:
    str: Success or error message.
    """
    try:
        # Construct the full table ID
        full_table_id = f"{dataset_id}.{table_id}"

        # Load the DataFrame to the BigQuery table
        to_gbq(df, full_table_id, project_id=project_id, if_exists=if_exists, progress_bar=True)

        # Optionally, check the number of rows in the BigQuery table
        client = bigquery.Client(project=project_id)
        query = f"SELECT COUNT(*) as total_rows FROM `{full_table_id}`"
        query_job = client.query(query)
        results = query_job.result()

        for row in results:
            total_rows = row.total_rows

        return f"Load successful. Total rows in the table now: {total_rows}"
    except Exception as e:
        return f"An error occurred: {str(e)}"

In [4]:
#load cpt_hcpcs.json into dataframe
# Set service account key
service_account_key_path = 'different-state-hospital-price-fd662d2f48c2.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account_key_path

# Specify JSON file path
bucket_path = 'gs://different-state-hospital-prices/cpt_hcpcs.json'

# Read JSON file into DataFrame
df_cpt_hcpcs = pd.read_json(bucket_path, lines=True)

print(df_cpt_hcpcs.info())
df_cpt_hcpcs.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287818 entries, 0 to 3287817
Data columns (total 3 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   code               object
 1   short_description  object
 2   long_description   object
dtypes: object(3)
memory usage: 75.3+ MB
None


,code,short_description,long_description
0,00000A,DVC REVASC 6X20MM 200CM,None
1,00001U,RBC DNA HEA 35 AG PLA,None
2,"00001U,1",RBC DNA HEA 35 AG PLA,None
3,00013,PT INDIVIDUAL GYM,None
4,0001A,HC ADM PFIZER SARSCOV2 30MCG/0.3ML 1ST,None


In [5]:
#cpt_hcpcs transformation
df_cpt_hcpcs = df_cpt_hcpcs.astype({
    'code': 'string',
    'short_description': 'string',
    'long_description': 'string'
})
df_cpt_hcpcs = df_cpt_hcpcs.fillna('None')

df_cpt_hcpcs = df_cpt_hcpcs.drop_duplicates()

print(df_cpt_hcpcs.info())
df_cpt_hcpcs.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287818 entries, 0 to 3287817
Data columns (total 3 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   code               string
 1   short_description  string
 2   long_description   string
dtypes: string(3)
memory usage: 75.3 MB
None


,code,short_description,long_description
0,00000A,DVC REVASC 6X20MM 200CM,None
1,00001U,RBC DNA HEA 35 AG PLA,None
2,"00001U,1",RBC DNA HEA 35 AG PLA,None
3,00013,PT INDIVIDUAL GYM,None
4,0001A,HC ADM PFIZER SARSCOV2 30MCG/0.3ML 1ST,None


In [6]:
#load hospitals.json into dataframe
bucket_path = 'gs://different-state-hospital-prices/hospitals.json'
df_hospitals = pd.read_json(bucket_path, lines=True)

print(df_hospitals.info())
df_hospitals.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   npi_number      1400 non-null   object 
 1   name            1400 non-null   object 
 2   url             1400 non-null   object 
 3   street_address  1351 non-null   object 
 4   city            1379 non-null   object 
 5   state           1377 non-null   object 
 6   zip_code        1357 non-null   object 
 7   publish_date    610 non-null    float64
dtypes: float64(1), object(7)
memory usage: 87.6+ KB
None


,npi_number,name,url,street_address,city,state,zip_code,publish_date
0,1003139775.0,HCA Virginia,https://hcavirginia.com/about/legal/pricing-tr...,901 E. Cary St Suite 210,Richmond,VA,None,1.609459e+12
1,1003260480,Brookwood Baptist Medical Center,https://www.brookwoodbaptisthealth.com/docs/gl...,2010 Brookwood Medical Center Dr.,Birmingham,AL,35209,NaN
2,1003281452,Henderson Hospital,https://uhsfilecdn.eskycity.net/ac/henderson-h...,1050 West Galleria Drive,Henderson,NV,89011,1.609459e+12
3,1003362997,CHI Health St. Elizabeth,https://www.chihealth.com/content/dam/chi-heal...,555 S. 70Th St.,Lincoln,NE,68510,1.609459e+12
4,1003389206,Merrill pioneer hospital,https://www.avera.org/app/files/public/79147/m...,"1100 S 10th Ave, Ste 100",Rock Rapids,IA,51246-2020,NaN


In [7]:
#hospitals transformation
#drop the uncessary column
df_hospitals = df_hospitals.drop(columns = ['publish_date'])

#Unify npi_number
df_hospitals['npi_number'] = df_hospitals['npi_number'].astype(str)
df_hospitals['npi_number'] = df_hospitals['npi_number'].str.replace(r'\D', '', regex=True)
df_hospitals['npi_number'] = df_hospitals['npi_number'].apply(lambda x: x[:10] if len(x) >= 10 else None).astype('string')

#unify zip_code
df_hospitals['zip_code'] = df_hospitals['zip_code'].astype('string')
df_hospitals['zip_code'] = df_hospitals['zip_code'].str.replace('.0','')

#Reference data for valid states
valid_states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

# Validates standard 5-digit or ZIP+4 formats
zip_code_pattern = r'^\d{5}(-\d{4})?$'

#Validates standard npi number
npi_number_pattern = r'\d{10}$'

# Filter DataFrame based on valid states and zip code pattern
df_hospitals = df_hospitals[df_hospitals['state'].isin(valid_states) & df_hospitals['zip_code'].str.match(zip_code_pattern)] & df_hospitals['npi_number'].str.match(npi_number_pattern)]

# covert other types using astype
df_hospitals = df_hospitals.astype({
    'name': 'string',
    'url': 'string',
    'street_address':'string',
    'city':'string',
    'state':'string',
})

df_hospitals = df_hospitals.fillna('None')

df_hospitals = df_hospitals.drop_duplicates(subset=['npi_number'])
print(df_hospitals.info())
df_hospitals.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1226 entries, 1 to 1399
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   npi_number      1226 non-null   string
 1   name            1226 non-null   string
 2   url             1226 non-null   string
 3   street_address  1226 non-null   string
 4   city            1226 non-null   string
 5   state           1226 non-null   string
 6   zip_code        1226 non-null   string
dtypes: string(7)
memory usage: 76.6 KB
None


,npi_number,name,url,street_address,city,state,zip_code
1,1003260480,Brookwood Baptist Medical Center,https://www.brookwoodbaptisthealth.com/docs/gl...,2010 Brookwood Medical Center Dr.,Birmingham,AL,35209
2,1003281452,Henderson Hospital,https://uhsfilecdn.eskycity.net/ac/henderson-h...,1050 West Galleria Drive,Henderson,NV,89011
3,1003362997,CHI Health St. Elizabeth,https://www.chihealth.com/content/dam/chi-heal...,555 S. 70Th St.,Lincoln,NE,68510
4,1003389206,Merrill pioneer hospital,https://www.avera.org/app/files/public/79147/m...,"1100 S 10th Ave, Ste 100",Rock Rapids,IA,51246-2020
5,1003811290,Providence Health,http://www.yourprovidencehealth.com/Content/Up...,2435 Forest Drive,Columbia,SC,29204


In [8]:
#load prices.json into dataframe
bucket_path = 'gs://different-state-hospital-prices/prices.json'

df_prices = pd.DataFrame()

# Create a JSON reader
json_reader = pd.read_json(bucket_path, lines=True, chunksize=10000)

# Process each chunk
for chunk in json_reader:
    # Clean 'npi_number' column
    chunk['npi_number'] = chunk['npi_number'].astype(str)
    chunk['npi_number'] = chunk['npi_number'].str.replace(r'\D', '', regex=True)
    chunk['npi_number'] = chunk['npi_number'].apply(lambda x: x[:10] if len(x) >= 10 else None).astype('string')

    # change dtype
    chunk = chunk.astype({
        'code': 'string',
        'payer': 'string'
    })
    
    # Append to dataframe
    df_prices = pd.concat([df_prices, chunk], ignore_index=True)
    
    print(len(df_prices))

df_prices = df_prices['npi_number'].str.match(npi_number_pattern)
    
df_prices = df_prices.fillna('none')

df_prices = df_prices.drop_duplicates()

print(df_prices.head())
print(df_prices.info())

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

KeyboardInterrupt: 

In [9]:
#parper for load
# Adding surrogate keys for each dimension table
df_cpt_hcpcs['cpt_id'] = np.arange(1, len(df_cpt_hcpcs) + 1)
df_hospitals['hospital_id'] = np.arange(1, len(df_hospitals) + 1)

# Mapping foreign keys in the fact table
df_prices['cpt_id'] = df_prices['code'].map(df_cpt_hcpcs.set_index('code')['cpt_id'])
df_prices['hospital_id'] = df_prices['npi_number'].map(df_hospitals.set_index('npi_number')['hospital_id'])

# Creating a unique list of payers and mapping payer_id
unique_payers = pd.DataFrame(df_prices['payer'].unique(), columns=['payer'])
unique_payers['payer_id'] = np.arange(1, len(unique_payers) + 1)
df_prices['payer_id'] = df_prices['payer'].map(unique_payers.set_index('payer')['payer_id'])

# Prepare final DataFrames for the new schema
dim_cpt_hcpcs = df_cpt_hcpcs[['cpt_id', 'code', 'short_description', 'long_description']]
dim_hospitals = df_hospitals[['hospital_id', 'npi_number', 'name', 'url', 'street_address', 'city', 'state', 'zip_code']]
dim_payer = unique_payers[['payer_id', 'payer']]
fact_prices = df_prices[['cpt_id', 'hospital_id', 'payer_id', 'price']]

print(dim_cpt_hcpcs.info())
print(dim_hospitals.info())
print(dim_payer.info())
print(fact_prices.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287818 entries, 0 to 3287817
Data columns (total 4 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   cpt_id             int32 
 1   code               string
 2   short_description  string
 3   long_description   string
dtypes: int32(1), string(3)
memory usage: 87.8 MB
None
<class 'pandas.core.frame.DataFrame'>
Index: 1226 entries, 1 to 1399
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   hospital_id     1226 non-null   int32 
 1   npi_number      1226 non-null   string
 2   name            1226 non-null   string
 3   url             1226 non-null   string
 4   street_address  1226 non-null   string
 5   city            1226 non-null   string
 6   state           1226 non-null   string
 7   zip_code        1226 non-null   string
dtypes: int32(1), string(7)
memory usage: 81.4 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5

In [10]:
# Load the DataFrame to the BigQuery table
result = load_dataframe_to_bigquery(dim_cpt_hcpcs, 'different_states_hospital_price', 'DimCptHcpcs', 'different-state-hospital-price')
print(result)


100%|██████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 962.00it/s]


Load successful. Total rows in the table now: 3287818
